# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846769530101                   -0.70    4.5
  2   -7.852306435112       -2.26       -1.53    1.0
  3   -7.852646195195       -3.47       -2.52    3.2
  4   -7.852646677583       -6.32       -3.36    2.2
  5   -7.852646685815       -8.08       -4.76    1.0
  6   -7.852646686722       -9.04       -5.19    3.5
  7   -7.852646686730      -11.15       -6.06    1.5
  8   -7.852646686730      -12.48       -7.16    2.5
  9   -7.852646686730   +    -Inf       -7.48    1.8
 10   -7.852646686730   +    -Inf       -8.51    2.0
 11   -7.852646686730      -14.75       -9.64    2.2
 12   -7.852646686730   +  -14.57       -9.82    2.0
 13   -7.852646686730      -14.57      -10.92    1.8
 14   -7.852646686730   +  -14.75      -12.20    2.8


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846847446690                   -0.70           4.8
  2   -7.852552617520       -2.24       -1.63   0.80    2.0
  3   -7.852640097629       -4.06       -2.71   0.80    1.0
  4   -7.852646446516       -5.20       -3.40   0.80    1.8
  5   -7.852646683322       -6.63       -4.28   0.80    2.0
  6   -7.852646686614       -8.48       -4.88   0.80    1.8
  7   -7.852646686721       -9.97       -5.81   0.80    1.5
  8   -7.852646686730      -11.06       -6.43   0.80    2.0
  9   -7.852646686730      -12.78       -7.25   0.80    1.5
 10   -7.852646686730      -14.27       -8.34   0.80    2.2
 11   -7.852646686730      -14.35       -8.96   0.80    2.5
 12   -7.852646686730   +  -15.05       -9.47   0.80    1.8
 13   -7.852646686730      -15.05      -10.39   0.80    1.5
 14   -7.852646686730   +  -15.05      -11.55   0.80    2.0
 15   -7.852646686730   +    -Inf      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.396375e+01     3.137870e+00
 * time: 0.41212892532348633
     1     1.168916e+00     1.653468e+00
 * time: 0.6133160591125488
     2    -1.426733e+00     1.840351e+00
 * time: 0.636955976486206
     3    -3.731734e+00     1.666868e+00
 * time: 0.7380950450897217
     4    -5.093792e+00     1.462161e+00
 * time: 0.771608829498291
     5    -6.755549e+00     9.035356e-01
 * time: 0.8054039478302002
     6    -6.884632e+00     1.408718e+00
 * time: 0.8286318778991699
     7    -7.380558e+00     1.087609e+00
 * time: 0.8516559600830078
     8    -7.408581e+00     1.355362e+00
 * time: 0.8750059604644775
     9    -7.562287e+00     1.122708e+00
 * time: 0.8983089923858643
    10    -7.731142e+00     3.553808e-01
 * time: 0.93190598487854
    11    -7.783562e+00     2.926766e-01
 * time: 0.955132007598877
    12    -7.813315e+00     7.241706e-02
 * time: 0.978693962097168
    13    -7.839681e+00     1.117169e-01
 * time: 1.001783847808838

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846813611051                   -0.70    4.5
  2   -7.852317407790       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686710                   -2.55
  2   -7.852646686730      -10.71       -5.94
  3   -7.852646686730   +  -15.05      -12.61


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.7489086601972053e-13
|ρ_newton - ρ_scfv| = 1.408752631141365e-13
|ρ_newton - ρ_dm|   = 1.0839011843988723e-9
